# Variables

In T-SQL we have the ability to use variables within our code.

These are objects we create in order to hold a single value of data that we can re-use throughout a batch or even reset within our code to make our SQL batches more procedural in nature.

This also allows for us to make our code much more generic, flexible, and easier to understand.

#### Declaring a Variable

In order to create a variable we must DECLARE it.  Unlike some other languages (such as Dim in Visual Basic and var in C#), we need to explicitly specify the datatype for our variable when we declare it and that can not change for the rest of the batch once declared.

Variables must be declared with an @ symbol prefixing our variable's name.

The format of this is as follows:

<code>declare @myVariable myDataType</code>

As an example we will create a variable called 'myID' which will store an Integer:

In [ ]:
declare @myID int;

It's as simple as that.  However this, in isolation, is not particularly useful as it has no value, therefore we should consider assigning a value to it.

There are several ways that we can do this and to start with we will focus on the two that are most commonly used, SELECT and SET.  For the purposes of assigning values to variables these work in an identical way:

In [ ]:
declare @myID int;

set @myID = 1;

In [ ]:
declare @myID int;

select @myID = 1;

Personally, between the two options I would tend to lean towards the latter, using SELECT as my keyword of choice.  The reason for this is nothing more than preference as I like the consistency of using SELECT throughout rather than adding a new keyword which adds nothing new or special.

However, I did mention that there were several ways to assign a variable and there is one more which I use quite a lot in order to make code more concise; the inline assignment:

In [ ]:
declare @myID int = 1;

This is a single statement creation and allocation and therefore cleaner, although all versions are perfectly valid and therefore use whichever allows you to feel more comfortable.

#### Using a Variable

Being able to declare and allocate a variable is not much use if we cannot use it for anything, therefore we will cover usage here.

Effectively a variable is an object storing a value and hence we can use it to replace hard coded values within our SQL statements.

Let's take this example, using the sales.basicSales table from a previous section:

In [ ]:
select *
from JupyterDatabase.sales.basicSales;

Using this table let's consider a simple example query that we can amend to use a variable:

In [ ]:
select *
from JupyterDatabase.sales.basicSales
where orderID = 7;

We can make this a little more generic by using a variable (this wouldn't be necessary in the real world for this simple example and a hard coded value would be better, this is purely to demonstrate usage and syntax):

In [ ]:
declare @orderID int = 7;

select *
from JupyterDatabase.sales.basicSales
where orderID = @orderID;

As you can see, this obtains the result that we require.  Again, it's basic and in this particular case it complicates a query for no valid reason, but in terms of syntax it acts as a good demonstration as to how we can declare and use a variable in a query.

To emphasise the syntax and how a variable works in another contrived example we can look at the following batch in which we create a variable, use the variable, and then we re-assign the value of the variable and use it again:

In [ ]:
declare @stringVariable varchar(100) = 'Fisher';

select *
from JupyterDatabase.sales.basicSales
where customerLName = @stringVariable;

select @stringVariable = 'Ann';

select *
from JupyterDatabase.sales.basicSales
where customerFName = @stringVariable;

In the above we have declared a variable as a varchar(100), this means it is a string and can be any number of characters from 0 to 100 in length.

First we assign it the value 'Fisher' and use it to look up all records with the customerLName of 'Fisher'.  We then re-assign the variable to be 'Ann' and look up all records where the customerFName is 'Ann'.

Hence we can re-assign the value of a variable as needed within a batch of statements.

Now you've seen the basics as to how these work we can move on to a more useful implementation of a variable.

Imagine we have a requirement that states two reports are required:
* Report One
 * We wish to see all sales for the last 5 years
 * We only wish to see the customer's full name and the sales amount
* Report Two
 * We wish to see the total of sales per year for the last 5 years

In our basic table we only have 1 sale per year and therefore this is a little redundant in terms of data, but the methods we can use are a valid real-world use case for variables and re-usable code.

One method of writing this report would be as follows:

In [ ]:
select concat_ws(' ', customerTitle, customerFName, customerLName) customerFullName, salesAmount
from JupyterDatabase.sales.basicSales
where year(orderDate) between 2015 and 2020;

select year(orderDate) orderYear, sum(salesAmount) totalSales
from JupyterDatabase.sales.basicSales
where year(orderDate) between 2015 and 2020
group by year(orderDate);

However, each year we would need to manually adjust the years in the query, which is not ideal and definitely not re-usable code.

This principle is also the case for reports covering week to date, month to date, year to date, yesterday etc. in which we have ever moving timeframes that we do not wish to have to manually adjust each morning.  It would therefore be ideal if we could have our SQL code take care of this for us.

We can do exactly this using variables:

In [ ]:
declare @startYear int = year(current_timestamp) - 5,
		@endYear int = year(current_timestamp);

select concat_ws(' ', customerTitle, customerFName, customerLName) customerFullName, salesAmount
from JupyterDatabase.sales.basicSales
where year(orderDate) between @startYear and @endYear;

select year(orderDate) orderYear, sum(salesAmount) totalSales
from JupyterDatabase.sales.basicSales
where year(orderDate) between @startYear and @endYear
group by year(orderDate);

Note we could have achieved this same result by not using variables but placing the calculations in each of the two statements.  However, this is why I chose to use two reports because you can see that the calculations would be duplicated in each report instead of declared once inside a variable.  Imagine we had a suite of ten reports all with duplicated calculations and then the calculation changed.  We would have to amend 10 reports and hope not to make any mistakes as opposed to changing just the variable declaration at the top and knowing that all queries will automatically change to our new logic accordingly.

For example, changing the report to the last 10 years would be much simpler in the variable version:

In [ ]:
declare @startYear int = year(current_timestamp) - 10,
		@endYear int = year(current_timestamp);

select concat_ws(' ', customerTitle, customerFName, customerLName) customerFullName, salesAmount
from JupyterDatabase.sales.basicSales
where year(orderDate) between @startYear and @endYear;

select year(orderDate) orderYear, sum(salesAmount) totalSales
from JupyterDatabase.sales.basicSales
where year(orderDate) between @startYear and @endYear
group by year(orderDate);

Lastly, notice how we can declare and assign multiple variables in one statement by separating them using a comma.

#### Assigning in a Query

Not only can we manually assign variables but we can also assign them using the output of a query.

In the following example we will assign a variable the total value of sales in our table and then return that value to our query window:

In [ ]:
declare @totalSales money;

select @totalSales = sum(salesAmount)
from JupyterDatabase.sales.basicSales;

select @totalSales;

The above scenario can be most useful if you wish to use queries to assign your variables that you then intend to use later in your batch.

For example, let's use a rudimentary method (there are better ways to achieve this but as a contrived example it serves a purpose nicely) to return only the record with the largest salesAmount:

In [ ]:
declare @salesAmount money;

select @salesAmount = max(salesAmount)
from JupyterDatabase.sales.basicSales;

select *
from JupyterDatabase.sales.basicSales
where salesAmount = @salesAmount;

You can therefore see how this can be utilised to store values within a query and re-used to retrieve data all in the same batch.

#### Variables and Batches

Note that I have been regularly specifying that the variables are relevant to a batch.  The reason for this is that variables are not stored anywhere except whilst your batch is running.

Each batch, as we learnt earlier, is a set of commands sent to SQL Server for execution.  Therefore if you declare a variable within one batch it is detroyed once that batch completes and so unavailable for the next batch to access.

To show this we will execute the following code all at once, it contains two batches (you can see from the 'go' batch separator) with the same statements in each batch except that the variables are only declared in the first batch:

In [ ]:
declare @totalSales money;

select @totalSales = sum(salesAmount)
from JupyterDatabase.sales.basicSales;

select @totalSales;
go

select @totalSales = sum(salesAmount)
from JupyterDatabase.sales.basicSales;

select @totalSales;
go

As the above demonstrates nicely, the original batch completes without issue, but the second batch fails with an error:

<code>Msg 137, Level 15, State 1, Line 75
Must declare the scalar variable "@totalSales".

Msg 137, Level 15, State 2, Line 78
Must declare the scalar variable "@totalSales".</code>

Hence a variable only lasts as long as a batch and internally SQL Server disposes of the variable the moment a batch ends (whether that be in success or failure).

If you wish for a variable to continue to be used then you must either ensure your statements are in the same batch or declare the variables again in each batch.